In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import logging
from pathlib import Path
from typing import Optional

import arviz as az
import numpyro

from hb_mep.config import HBMepConfig
from hb_mep.data_access import DataClass
# from hb_mep.models.rats import MixtureModel
from hb_mep.models.rats.utils import load_data
from hb_mep.utils import plot

numpyro.set_platform("cpu")
numpyro.set_host_device_count(12)
numpyro.enable_x64()

PARENT_PATH = Path(os.getcwd()).parent.parent.absolute()
FORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)


import logging
from typing import Optional

import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive
from numpyro.diagnostics import hpdi

from hb_mep.config import HBMepConfig
from hb_mep.models.baseline import Baseline
from hb_mep.models.utils import Site as site
from hb_mep.utils import timing
from hb_mep.utils.constants import (
    INTENSITY,
    RESPONSE,
    PARTICIPANT,
    FEATURES
)

#### Load config

In [3]:
config = HBMepConfig()
config.CURRENT_PATH = PARENT_PATH

#### Load data and preprocess

In [4]:
data = DataClass(config)

""" Load """
dir_name = "physio2"
dir = os.path.join(data.data_path, dir_name)
participants = range(1, 9)
subdir_pattern = ["*L_CIRC*"]

df, mat, time = load_data(
    dir=dir, participants=participants, subdir_pattern=subdir_pattern
)

""" Preprocess """
df, encoder_dict, mat = data.build(df=df, mat=mat)

2023-06-26 15:47:22,771 - hb_mep.utils.utils - INFO - func:load_data took: 1.32 sec
2023-06-26 15:47:22,772 - hb_mep.data_access.core - INFO - Processing data ...
2023-06-26 15:47:22,853 - hb_mep.utils.utils - INFO - func:preprocess took: 0.08 sec
2023-06-26 15:47:22,854 - hb_mep.utils.utils - INFO - func:build took: 0.08 sec


In [5]:
save_path = os.path.join(data.reports_path, "traces_current.pdf")
plot(
    df=df,
    save_path=save_path,
    encoder_dict=encoder_dict,
    mat=mat,
    time=time
);